In [166]:
import pandas as pd

In [167]:
my_protein = pd.read_csv("myprotein_nutrition.csv")
my_protein.head()

,store,product_name,product_url,price_total,raw_price_text,n_reviews,serving_size,calories,protein_g,serving_size_g,rating_avg,Number of Servings
0,myprotein,Myprotein Clear Whey Isolate - MARVEL,https://us.myprotein.com/p/sports-nutrition/my...,39.99,$39.99â,29,1 scoop (25g),80.0,20.0,25.0,4.17,20servings
1,myprotein,Impact Whey Protein,https://us.myprotein.com/p/sports-nutrition/im...,19.99,$19.99â,4389,1 scoop (25g),100.0,19.0,25.0,4.48,0.55lb
2,myprotein,Clear Whey Isolate,https://us.myprotein.com/p/sports-nutrition/cl...,39.99,$39.99â,899,1 scoop (25g),80.0,20.0,25.0,4.37,1.1lb
3,myprotein,Clear Whey Isolate (Sample),https://us.myprotein.com/p/sports-nutrition/cl...,3.99,$3.99â,53,1 scoop (25g),80.0,20.0,25.0,4.45,0.88Oz
4,myprotein,Pea Protein Isolate,https://us.myprotein.com/p/sports-nutrition/pe...,34.99,$34.99â,279,1 Large Scoop (25g),100.0,21.0,25.0,3.70,2.2lb


dont need carbs and fat

In [168]:
my_protein = my_protein.drop(columns=["raw_price_text"])
my_protein["store"] = my_protein["store"].replace("myprotein", "Myprotein")

drop all bundles

In [169]:
my_protein = my_protein[~my_protein["product_name"].str.contains("bundle", case=False, na=False)]

In [170]:
# Updated dict of manual imputations: index -> {column: value, ...}
impute_map = {
    24: {"rating_avg": 5.0, "calories": 110},
    30: {"price_total": 54.99},
    22: {"rating_avg":5.0}
}

# Apply imputations
for idx, updates in impute_map.items():
    for col, val in updates.items():
        my_protein.loc[idx, col] = val

In [171]:
my_protein = my_protein[my_protein["calories"].notna()]
my_protein = my_protein[my_protein["serving_size_g"].notna()]

In [172]:
my_protein.isnull().sum()

store                 0
product_name          0
product_url           0
price_total           0
n_reviews             0
serving_size          0
calories              0
protein_g             0
serving_size_g        0
rating_avg            0
Number of Servings    0
dtype: int64

In [173]:
rename_map = {
    "store": "Store",
    "product_name": "Product Name",
    "product_url": "Product Link",
    "price_total": "Price",
    "n_reviews": "Num Reviews",
    "rating_avg": "Avg Rating",
    "calories": "Calories",
    "protein_g": "Protein (g)",
    "Number of Servings": "Total Num Servings",
    "serving_size": "Serving Size",
    "serving_size_g": "Serving Size (g)",
}

my_protein = my_protein.rename(columns=rename_map)

desired_order = [
    "Store",
    "Product Name",
    "Product Link",
    "Price",
    "Num Reviews",
    "Avg Rating",
    "Calories",
    "Protein (g)",
    "Total Num Servings",
    "Serving Size",
    "Serving Size (g)"
]

my_protein = my_protein[desired_order]

my_protein

,Store,Product Name,Product Link,Price,Num Reviews,Avg Rating,Calories,Protein (g),Total Num Servings,Serving Size,Serving Size (g)
0,Myprotein,Myprotein Clear Whey Isolate - MARVEL,https://us.myprotein.com/p/sports-nutrition/my...,39.99,29,4.17,80.0,20.0,20servings,1 scoop (25g),25.0
1,Myprotein,Impact Whey Protein,https://us.myprotein.com/p/sports-nutrition/im...,19.99,4389,4.48,100.0,19.0,0.55lb,1 scoop (25g),25.0
2,Myprotein,Clear Whey Isolate,https://us.myprotein.com/p/sports-nutrition/cl...,39.99,899,4.37,80.0,20.0,1.1lb,1 scoop (25g),25.0
3,Myprotein,Clear Whey Isolate (Sample),https://us.myprotein.com/p/sports-nutrition/cl...,3.99,53,4.45,80.0,20.0,0.88Oz,1 scoop (25g),25.0
4,Myprotein,Pea Protein Isolate,https://us.myprotein.com/p/sports-nutrition/pe...,34.99,279,3.70,100.0,21.0,2.2lb,1 Large Scoop (25g),25.0
5,Myprotein,Whey Forward,https://us.myprotein.com/p/sports-nutrition/wh...,29.99,165,3.87,90.0,20.0,20servings,1 Scoop (25.2g),25.2
6,Myprotein,Impact Whey Isolate,https://us.myprotein.com/p/sports-nutrition/im...,29.99,2427,4.38,110.0,25.0,0.55lb,1 Scoop (29g),29.0
7,Myprotein,Clear Whey MIKE AND IKEÂ® Flavors,https://us.myprotein.com/p/sports-nutrition/cl...,39.99,210,4.52,90.0,20.0,1.1lb - 20servings,1 Scoop (26.3g),26.3
9,Myprotein,Myprotein Clear Whey Isolate - MARVEL (Sample),https://us.myprotein.com/p/sports-nutrition/my...,3.99,5,3.40,80.0,20.0,1servings,1 Sachet (25g),25.0
10,Myprotein,Weight Gainer Blend,https://us.myprotein.com/p/sports-nutrition/we...,54.99,87,3.63,365.0,27.0,5.5lb,3 1/3 large scoops (100g),100.0


In [175]:
my_protein.shape

(25, 11)

In [176]:
my_protein.to_csv("myprotein_final.csv", index=False)